# 05. Обучение свёрточных нейронных сетей (CNN)

## План
1. Базовый подход на датасете CIFAR-10
2. Добавление аугментаций данных
3. Использование предобученных моделей (Fine-tuning)
4. Стратегии планирования скорости обучения

 ### 0. Настройка Wandb для логирования экспериментов

In [ ]:
! pip install wandb -q

In [ ]:
import wandb

wandb.login()

In [ ]:
%config InlineBackend.figure_format='retina'

 ## 1. Базовый подход на датасете CIFAR-10

На предыдущих занятиях мы освоили построение базовых архитектур нейронных сетей и их обучение на различных наборах изображений.
В рамках данного семинара мы рассмотрим методы улучшения качества моделей за счёт настройки различных параметров.

Ключевые аспекты, влияющие на качество модели:

* **Архитектурные решения**
  * Выбор семейства архитектур (**ResNet**, **EfficientNet** и др.)
  * Размер модели (**ResNet18** vs **ResNet101**)
  * Количество обучаемых слоев
  * Стратегия постепенного размораживания слоев

* **Параметры оптимизации**
  * Алгоритм оптимизации (**SGD**, **Adam** и их варианты)
  * Скорость обучения и ее изменение в процессе обучения
  * Добавление момента
  * Регуляризация (weight decay)
  
* **Обработка данных**
  * Взвешивание классов и стратегии семплирования
  * Набор аугментаций и их интенсивность
  * Предобработка и очистка данных

* **Параметры обучения**
  * Размер мини-пакета
  * Функция потерь
  * Критерии оценки (помимо val_loss)
  * Условия остановки обучения


### 1.1. Загрузка и подготовка данных

In [ ]:
import os
import glob
import pickle
import tqdm
import cv2
from torchvision.datasets import CIFAR10

### 1.2. Создание датасета и анализ данных

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T

Ранее мы обрабатывали изображения непосредственно в датасете. 

Рекомендуется использовать механизм трансформаций для стандартизации предобработки данных:

In [ ]:
transforms_simple = T.Compose(
    [
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])
    ]  # ⬆︎ — "стандарт по-умолчанию" для быстрого старта, в реальных задачах берём из набора данных
)
# normalized_tensor = (tensor - mean) / std

Для трансформаций данных мы используем модуль `torchvision.transforms`, однако существуют и альтернативные библиотеки (которые будут рассмотрены далее).

На текущем этапе ограничимся базовыми операциями преобразования - конвертацией в тензор и нормализацией.

In [ ]:
dataset_train = CIFAR10("./", train=True, download=True, transform=transforms_simple)

In [ ]:
image, label = dataset_train[0]
image.shape, label

In [ ]:
cifar10_class_map = {idx: idx_class for idx_class, idx in dataset_train.class_to_idx.items()}
cifar10_class_map

Для визуализации нормализованных тензоров необходимо выполнить обратное преобразование — денормализацию:

**Задача**: реализовать функцию `tensor_to_image`, преобразующую нормализованный тензор размерности `(3, h, w)` в денормализованный массив `(h, w, 3)`.

In [ ]:
def tensor_to_image(tensor, mean=(0.5, 0.5, 0.5), std=(0.25, 0.25, 0.25)):
    ### YOUR CODE HERE

    # mean, std -> ...

    ### END OF YOUR CODE

    return image

In [ ]:
image = np.random.uniform(size=(32, 32, 3))
tensor = (torch.from_numpy(image).permute(2, 0, 1) - 0.5) / 0.25

np.testing.assert_array_equal(image, tensor_to_image(tensor))

Посмотрим глазами на данные:

In [ ]:
indexes_to_show = np.random.choice(len(dataset_train), size=64, replace=False)

plt.figure(figsize=(18, 14))
for i, index in enumerate(indexes_to_show):
    tensor, label = dataset_train[index]
    image = tensor_to_image(tensor)
    plt.subplot(8, 8, i + 1)
    plt.imshow(image)
    plt.axis(False)
    plt.title(f"GT: {label} ({cifar10_class_map[label]})")
plt.show()

Стандартной практикой является проведение разведывательного анализа данных (Exploratory Data Analysis, EDA). 

На данном этапе ограничимся анализом распределения количества изображений по классам.

**Задача**: используя любой удобный метод, определить количество изображений для каждого класса в обучающей выборке.

In [ ]:
labels_train = np.array([label for _, label in dataset_train])

In [ ]:
### YOUR CODE HERE

### END OF YOUR CODE

Создадим валидационный набор данных и перейдем к следующему этапу:

In [ ]:
dataset_val = CIFAR10("./", train=False, download=False, transform=transforms_simple)

In [ ]:
labels_val = np.array([label for _, label in dataset_val])

In [ ]:
### YOUR CODE HERE

### END OF YOUR CODE

### 1.3. Построение модели

Начнем с базовой архитектуры, которую можно быстро реализовать и протестировать.
В дальнейшем в качестве базового решения можно рассматривать ResNet34, но для начала реализуем простую архитектуру.

**Задание**: реализовать класс CNNBlock для построения блока свёрточной сети со следующей структурой:
* Сверточный слой 3x3
* Пакетная нормализация
* ReLU активация
* Сверточный слой 3x3
* Пакетная нормализация
* ReLU активация
* (Опционально) MaxPooling 2x2

In [ ]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pool=True):
        super(CNNBlock, self).__init__()

        ### YOUR CODE HERE
        
        ### END OF YOUR CODE

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x

Соберём из этих блоков сеть:

In [ ]:
cnn_baseline = nn.Sequential(
    CNNBlock(3, 32),
    CNNBlock(32, 64),
    CNNBlock(64, 128),
    CNNBlock(128, 256),
    CNNBlock(256, 512),
    # v NOTE THIS
    nn.AdaptiveAvgPool2d((1, 1)),  # B x 512 x 1 x 1
    # ^ NOTE THIS ^
    nn.Flatten(),  # B x 512
    nn.Linear(512, 10),
).eval()

In [ ]:
x = torch.randn(4, 3, 32, 32)
y = cnn_baseline(x)
y.shape

### 1.4. Процесс обучения
#### 1.4.1 Настройка логирования с использованием Wandb

In [ ]:
len(dataset_train)

In [ ]:
import wandb

config = {
    "learning_rate": 3e-4,
    "weight_decay": 0.01,
    "batch_size": 50,
    "num_epochs": 5,
    "optimizer": torch.optim.AdamW,
}

wandb.init(
    # set the wandb project where this run will be logged
    project="seminar-05-cnn-2025",
    name="baseline",
    reinit=True,
    # track hyperparameters and run metadata
    config=config,
)

In [ ]:
# Автоматический выбор устройства с приоритетом: MPS -> CUDA -> CPU
device = torch.device(
    "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else
    "cuda" if torch.cuda.is_available() else 
    "cpu"
)

print(f"Используемое устройство: {device}")

In [ ]:
import os

num_workers = os.cpu_count()
num_workers

In [ ]:
dataloader_train = DataLoader(
    dataset_train,
    batch_size=config["batch_size"],
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=4,
)

dataloader_val = DataLoader(
    dataset_val,
    batch_size=config["batch_size"],
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=4,
)

Инициализируйте функцию потерь и оптимизатор Adam, используя соответствующие модули PyTorch:

In [ ]:
config

In [ ]:
### YOUR CODE HERE

# loss_fn = ...
# optimizer = ...

### END OF YOUR CODE

Ранее мы использовали раздельные методы для этапов обучения и валидации. Теперь объединим их в единую функцию:

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):
    # v NOTE THIS v
    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
    else:
        torch.set_grad_enabled(False)
        model.eval()
    # ^ NOTE THIS ^

    model = model.to(device)
    num_steps = len(dataloader)
    losses = []
    for i, batch in enumerate(tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}")):
        xs, ys_true = batch

        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # логгируем номер шага при обучении
            wandb.log({
                "training_step": i + num_steps * epoch,
                "lr": optimizer.param_groups[0]["lr"],
                "loss": loss,
            })

        losses.append(loss.detach().cpu().item())

    return np.mean(losses)

При вызове метода `wandb.log` происходит обновление внутреннего счетчика шагов системы логирования, который не зависит от номера итерации обучения или валидации. 
Для обеспечения корректного сравнения различных метрик рекомендуется совместно записывать значения функций потерь, метрик качества, а также номер шага обучения или эпохи.

Кроме того, мы переходим к более продвинутой стратегии управления процессом обучения, чем простое сохранение последнего состояния модели.
Будем отслеживать значение целевой метрики (в данном случае `val_loss`) и сохранять контрольные точки модели только при достижении наилучших результатов.

In [ ]:
def save_checkpoint(model, filename):
    with open(filename, "wb") as fp:
        torch.save(model.state_dict(), fp)

def load_checkpoint(model, filename):
    with open(filename, "rb") as fp:
        state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(state_dict)

In [ ]:
my_model = nn.Linear(100, 1)
my_model.weight.data *= 1e6
save_checkpoint(my_model, "test.pth.tar")

my_model_new = nn.Linear(100, 1)
load_checkpoint(my_model_new, "test.pth.tar")

torch.testing.assert_allclose(my_model.weight, my_model_new.weight)

Учитывая планируемое количество экспериментов, для избежания дублирования кода инкапсулируем всю логику процесса обучения в функцию `run_experiment()`:

In [ ]:
def run_experiment(
    model, dataloader_train, dataloader_val, loss_fn, optimizer, num_epochs, device, output_dir, start_epoch=0
):
    train_losses = []
    val_losses = []

    best_val_loss = np.inf
    best_val_loss_epoch = -1
    best_val_loss_fn = None

    os.makedirs(output_dir, exist_ok=True)

    for epoch in range(start_epoch, num_epochs):
        train_loss = run_epoch("train", model, dataloader_train, loss_fn, optimizer, epoch, device)
        train_losses.append(train_loss)

        val_loss = run_epoch("val", model, dataloader_val, loss_fn, optimizer, epoch, device)
        val_losses.append(val_loss)

        wandb.log({"epoch_loss_train": train_loss, "epoch_loss_val": val_loss, "epoch": epoch})

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_loss_epoch = epoch

            output_fn = os.path.join(output_dir, f"epoch={str(epoch).zfill(2)}_valloss={best_val_loss:.3f}.pth.tar")
            save_checkpoint(model, output_fn)
            print(f"New checkpoint saved to {output_fn}\n")

            best_val_loss_fn = output_fn

    print(f"Best val_loss = {best_val_loss:.3f} reached at epoch {best_val_loss_epoch}")
    load_checkpoint(model, best_val_loss_fn)

    return train_losses, val_losses, best_val_loss, model

Запустим:

In [ ]:
train_losses_baseline, val_losses_baseline, best_val_loss_baseline, cnn_baseline = run_experiment(
    cnn_baseline,
    dataloader_train,
    dataloader_val,
    loss_fn,
    optimizer,
    config["num_epochs"],
    device,
    "checkpoints_baseline",
)

In [ ]:
wandb.finish()

Анализ результатов обучения:

In [ ]:
def plot_losses(train_losses, val_losses, title):
    plt.figure(figsize=(12, 5))
    plt.title(title)
    plt.plot(train_losses, label="train")
    plt.plot(val_losses, label="val")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
plot_losses(train_losses_baseline, val_losses_baseline, title="cnn_baseline")

Вычисление метрик качества:

In [ ]:
def collect_predictions(model, dataloader, device):
    model.eval()
    model = model.to(device)
    torch.set_grad_enabled(False)

    labels_all = []
    probs_all = []
    preds_all = []
    for batch in tqdm.tqdm(dataloader, total=len(dataloader)):
        images, labels = batch

        logits = model(images.to(device))
        # softmax вычислительно неэффективен, поэтому не стоит лишний раз им пользоваться
        # probs = logits.softmax(dim=1)
        probs = logits.to(device="cpu")
        max_prob, max_prob_index = torch.max(probs, dim=1)

        labels_all.extend(labels.numpy().tolist())
        probs_all.extend(max_prob.numpy().tolist())
        preds_all.extend(max_prob_index.numpy().tolist())

    return labels_all, probs_all, preds_all

In [ ]:
train_labels, train_probs, train_preds = collect_predictions(cnn_baseline, dataloader_train, device)

accuracy_train = accuracy_score(train_labels, train_preds)
accuracy_train

In [ ]:
train_labels[:5], train_preds[:5], train_probs[:5]

In [ ]:
val_labels, val_probs, val_preds = collect_predictions(cnn_baseline, dataloader_val, device)

accuracy_val = accuracy_score(val_labels, val_preds)
accuracy_val

In [ ]:
val_labels[:5], val_probs[:5], val_preds[:5]

## 2. Применение аугментаций данных

Аугментация данных представляет собой один из фундаментальных методов повышения обобщающей способности моделей. 
Для реализации аугментаций можно использовать как встроенный модуль `torchvision.transforms`, так и специализированные библиотеки, например, [`albumentations`](https://albumentations.ai/). 
Существуют также [автоматические методы аугментации](https://pytorch.org/vision/main/generated/torchvision.transforms.AutoAugment.html), выходящие за рамки данного семинара.

Чрезмерно агрессивные аугментации могут негативно сказаться на процессе обучения, поэтому начнем с умеренных преобразований:

In [ ]:
config["transforms"] = T.Compose(
    [
        T.ToTensor(),
        ### YOUR CODE HERE
        
        ### END OF YOUR CODE
        T.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25]),
    ]
)

In [ ]:
dataset_aug_train = CIFAR10("./", train=True, download=False, transform=config["transforms"])

In [ ]:
indexes_to_show = np.random.choice(len(dataset_aug_train), size=64, replace=False)

plt.figure(figsize=(18, 14))
for i, index in enumerate(indexes_to_show):
    tensor, label = dataset_aug_train[index]
    image = tensor_to_image(tensor)
    plt.subplot(8, 8, i + 1)
    plt.imshow(image)
    plt.axis(False)
    plt.title(f"GT: {label} ({cifar10_class_map[label]})")
plt.show()

Оценим эффект от применения аугментаций, сохраняя валидационный набор данных без изменений:

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32),
    CNNBlock(32, 64),
    CNNBlock(64, 128),
    CNNBlock(128, 256),
    CNNBlock(256, 512),
    # v NOTE THIS
    nn.AdaptiveAvgPool2d((1, 1)),  # B x 512 x 1 x 1
    # ^ NOTE THIS ^
    nn.Flatten(),  # B x 512
    nn.Linear(512, 10),
).eval()

Увеличим количество эпох обучения, поскольку применение аугментаций позволяет снизить степень переобучения и обеспечивает целесообразность дополнительных итераций обучения.

In [ ]:
dataloader_aug_train = DataLoader(
    dataset_aug_train,
    batch_size=config["batch_size"],
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=4,
)

dataloader_val = DataLoader(
    dataset_val,
    batch_size=config["batch_size"],
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=4,
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = config["optimizer"](cnn_aug.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="seminar-05-cnn-2025",
    name="baseline_augs",
    reinit=True,
    # track hyperparameters and run metadata
    config=config,
)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, config["num_epochs"], device, "checkpoints_aug"
)

In [ ]:
plot_losses(train_losses_aug, val_losses_aug, title="cnn_aug")

In [ ]:
train_labels, train_probs, train_preds = collect_predictions(cnn_aug, dataloader_aug_train, device)

accuracy_train = accuracy_score(train_labels, train_preds)
accuracy_train

In [ ]:
val_labels, val_probs, val_preds = collect_predictions(cnn_aug, dataloader_val, device)

accuracy_val = accuracy_score(val_labels, val_preds)
accuracy_val

Анализ функций потерь и метрик качества на валидационной выборке демонстрирует снижение степени переобучения модели и указывает на необходимость увеличения количества итераций для достижения сходимости.

В рамках самостоятельной работы рекомендуется дополнить систему логирования метриками качества для более комплексной оценки результатов.

## 3. Использование предобученных моделей (Fine-tuning)

Использование предобученных моделей представляет собой стандартную практику в современном глубоком обучении.

В большинстве случаев инициализация весов предобученной моделью демонстрирует более высокую эффективность по сравнению со случайной инициализацией, особенно когда исходная модель обучалась на домене, родственном целевому.

Доступные источники предобученных моделей включают:
* [`torchvision.models`](https://pytorch.org/vision/0.8/models.html)
* [`timm`](https://github.com/rwightman/pytorch-image-models)
* [`transformers`](https://github.com/huggingface/transformers)


In [ ]:
from torchvision import models as M

Архитектура ResNet является фундаментальной в области компьютерного зрения. В рамках эксперимента мы используем 18-слойную модификацию данной архитектуры:

![Схема архитектуры ResNet-18](https://velog.velcdn.com/images%2Fe_sin528%2Fpost%2Fe272c056-3dfa-4bb6-bfc9-b309d82df932%2FResNet18.png)

In [ ]:
resnet18 = M.resnet18(pretrained=True)

In [ ]:
import torchvision

In [ ]:
torchvision.models.ResNet18_Weights.DEFAULT

In [ ]:
resnet18.eval();

In [ ]:
resnet18

In [ ]:
x = torch.randn(1, 3, 224, 224)

y = resnet18(x)
y.size()

**Стратегии адаптации предобученных моделей**

Для интеграции предобученных моделей в целевые задачи применяются следующие подходы:

* **Модификация классификатора** - замена выходного полносвязного слоя на слой с количеством нейронов, соответствующим числу классов целевой задачи
   * Библиотека `timm` предоставляет встроенные механизмы для данной модификации при инициализации модели

* **Использование экстрактора признаков** - применение convolutional backbone модели в качестве фиксированного экстрактора признаков с последующим добавлением пользовательских слоев для классификации
   * Библиотека `timm` также упрощает реализацию данного подхода

Особенность нашего случая заключается в том, что стандартная архитектура ResNet18 предполагает уменьшение пространственного разрешения входного изображения в 64 раза, тогда как размер изображений в нашем датасете составляет 32×32 пикселя. Это создает технические сложности, требующие специальных решений для адаптации модели.

In [ ]:
x = torch.randn(1, 3, 32, 32)

y = resnet18(x)
y.size()

Один из **распространенных подходов** предполагает *декомпозицию* предобученной модели на **структурные компоненты** с последующим *формированием* новой архитектуры на основе `nn.Sequential` и *интеграцией* дополнительных **пользовательских слоев**:

In [ ]:
resnet18

In [ ]:
cnn_finetuned = nn.Sequential(
    resnet18.conv1,
    resnet18.bn1,
    resnet18.relu,
    resnet18.maxpool,
    resnet18.layer1,
    resnet18.layer2,
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(128, 10),
).eval()

In [ ]:
cnn_finetuned(x)

**Стратегия обучения** модели требует особого подхода. Наличие *частично обученных весов* в начальных слоях (из ResNet) и *полностью необученных весов* в выходных слоях создает ситуацию, когда **градиенты в конечных слоях** могут характеризоваться *высоким уровнем шума*. 

Для решения этой проблемы применяется **двухэтапная процедура обучения**: на первом этапе осуществляется *обучение исключительно новых головных слоев* в течение нескольких эпох, после чего выполняется *полное обучение* всей архитектуры:

In [ ]:
cnn_finetuned[0].weight.requires_grad, cnn_finetuned[-1].weight.requires_grad

**Управление обучаемостью параметров** модели может быть осуществлено *вручную* через механизм **заморозки весов** слоев:

In [ ]:
for p in cnn_finetuned.parameters():
    p.requires_grad_(False)

In [ ]:
cnn_finetuned[0].weight.requires_grad, cnn_finetuned[-1].weight.requires_grad

In [ ]:
cnn_finetuned[-1].requires_grad_(True)
cnn_finetuned[-1].weight.requires_grad

In [ ]:
cnn_finetuned[-1]

In [ ]:
[(n, p.requires_grad) for (n, p) in cnn_finetuned.named_parameters()]

**Переходим к этапу обучения модели:**

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = config["optimizer"](
    cnn_finetuned[-1].parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"]
)

**На первом этапе** выполняем обучение *исключительно последнего слоя* в течение **трех эпох**:

In [ ]:
wandb.finish()

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="seminar-05-cnn-2025",
    name="resnet",
    reinit=True,
    # track hyperparameters and run metadata
    config=config,
)

In [ ]:
train_losses_finetuned, val_losses_finetuned, best_val_loss_finetuned, cnn_finetuned = run_experiment(
    cnn_finetuned, dataloader_aug_train, dataloader_val, loss_fn, optimizer, 3, device, "checkpoints_finetuned"
)

**На втором этапе** выполняем *полную разморозку сети* и осуществляем *обучение всей архитектуры*:

In [ ]:
for p in cnn_finetuned.parameters():
    p.requires_grad_(True)


optimizer = config["optimizer"](
    cnn_finetuned.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"]
)

In [ ]:
train_losses_finetuned, val_losses_finetuned, best_val_loss_finetuned, cnn_finetuned = run_experiment(
    cnn_finetuned,
    dataloader_aug_train,
    dataloader_val,
    loss_fn,
    optimizer,
    config["num_epochs"],
    device,
    "checkpoints_finetuned",
    start_epoch=3,
)

In [ ]:
plot_losses(train_losses_finetuned, val_losses_finetuned, title="cnn_finetuned")

In [ ]:
train_labels, train_probs, train_preds = collect_predictions(cnn_finetuned, dataloader_aug_train, device)

accuracy_train = accuracy_score(train_labels, train_preds)
accuracy_train

In [ ]:
val_labels, val_probs, val_preds = collect_predictions(cnn_finetuned, dataloader_val, device)

accuracy_val = accuracy_score(val_labels, val_preds)
accuracy_val

In [ ]:
wandb.finish()

## 4. Стратегии планирования скорости обучения

**Завершающий аспект** нашего рассмотрения - *управление скоростью обучения* (Learning Rate).

Как следует из материалов лекций, *динамическое изменение скорости обучения* в процессе тренировки (включая использование адаптивных методов оптимизации) способствует достижению **более высокого качества модели**.

# ![Сравнение стратегий изменения скорости обучения](https://i.stack.imgur.com/UHYMw.png)

Фреймворк `PyTorch` предоставляет *комплексный инструментарий* для реализации различных стратегий *управление скоростью обучения* (learning rate).

### 4.1. Адаптивное изменение скорости обучения на основе метрик (ReduceLROnPlateau)

**Метод адаптивного управления** скорость обучения (learning rate) основывается на *мониторинге изменений целевых показателей*.

Например, при *стагнации функции потерь* на протяжении нескольких эпох, рекомендуется **уменьшение значения скорости обучения (learning rate)**:

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32), CNNBlock(32, 64), CNNBlock(64, 128), nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(128, 10)
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
config["learning_rate"] = 1.5

optimizer = torch.optim.Adam(cnn_aug.parameters(), lr=config["learning_rate"])

scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=1, verbose=True)

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="seminar-05-cnn-2025",
    name="lr_reduce",
    reinit=True,
    # track hyperparameters and run metadata
    config=config,
)

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):
    # v NOTE THIS v
    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
    else:
        torch.set_grad_enabled(False)
        model.eval()
    # ^ NOTE THIS ^

    model = model.to(device)
    num_steps = len(dataloader)
    losses = []
    for i, batch in enumerate(tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}")):
        xs, ys_true = batch

        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # логгируем номер шага при обучении
            wandb.log({
                "training_step": i + num_steps * epoch,
                "lr": optimizer.param_groups[0]["lr"],
                "loss": loss,
            })

        losses.append(loss.detach().cpu().item())

    if stage != "train":
        scheduler.step(np.mean(losses))

    return np.mean(losses)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, config["num_epochs"], device, "checkpoints_aug"
)

In [ ]:
wandb.finish()

### 4.2. Поэтапное изменение скорости обучения на каждой эпохе (StepLR)

In [ ]:
from torch.optim.lr_scheduler import StepLR

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32), CNNBlock(32, 64), CNNBlock(64, 128), nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(128, 10)
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
config["learning_rate"] = 3e-4
config["num_epochs"] = 5

optimizer = torch.optim.Adam(cnn_aug.parameters(), lr=config["learning_rate"])

scheduler = StepLR(optimizer, step_size=1, gamma=0.1, verbose=True)

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="seminar-05-cnn-2025",
    name="lr_step",
    reinit=True,
    # track hyperparameters and run metadata
    config=config,
)

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):
    # v NOTE THIS v
    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
    else:
        torch.set_grad_enabled(False)
        model.eval()
    # ^ NOTE THIS ^

    model = model.to(device)
    num_steps = len(dataloader)
    losses = []
    for i, batch in enumerate(tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}")):
        xs, ys_true = batch

        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # логгируем номер шага при обучении
            wandb.log({
                "training_step": i + num_steps * epoch,
                "lr": optimizer.param_groups[0]["lr"],
                "loss": loss,
            })

        losses.append(loss.detach().cpu().item())

    if stage != "train":
        scheduler.step()

    return np.mean(losses)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, config["num_epochs"], device, "checkpoints_aug"
)

In [ ]:
wandb.finish()

### 4.3. Пошаговое изменение скорости обучения на каждой итерации (CosineAnnealingLR)

**Арсенал методов** регулировки скорости обучения включает подходы, основанные на *аналитических зависимостях*. 

Примером такой стратегии является алгоритм **[CosineAnnealing](https://paperswithcode.com/method/cosine-annealing)**, использующий косинусоидальную функцию для плавного изменения learning rate.

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32), CNNBlock(32, 64), CNNBlock(64, 128), nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(128, 10)
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
config["learning_rate"] = 3e-4
config["num_epochs"] = 5

optimizer = torch.optim.Adam(cnn_aug.parameters(), lr=config["learning_rate"])

scheduler = CosineAnnealingLR(optimizer, T_max=int(len(dataloader_aug_train)) * config["num_epochs"])

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="seminar-05-cnn-2025",
    name="lr_cosine",
    reinit=True,
    # track hyperparameters and run metadata
    config=config,
)

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):
    # v NOTE THIS v
    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
    else:
        torch.set_grad_enabled(False)
        model.eval()
    # ^ NOTE THIS ^

    model = model.to(device)
    num_steps = len(dataloader)
    losses = []
    for i, batch in enumerate(tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}")):
        xs, ys_true = batch

        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # логгируем номер шага при обучении
            wandb.log({
                "training_step": i + num_steps * epoch,
                "lr": optimizer.param_groups[0]["lr"],
                "loss": loss,
            })
            # вызываем после каждого шага оптимизации
            scheduler.step()

        losses.append(loss.detach().cpu().item())

    return np.mean(losses)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, config["num_epochs"], device, "checkpoints_aug"
)

In [ ]:
wandb.finish()

## Итоги занятия

В рамках данного семинара были рассмотрены следующие ключевые аспекты:

* **Применение аугментаций данных** для улучшения обобщающей способности моделей
* **Методы использования предобученных моделей** и стратегии их адаптации к целевым задачам
* **Реализация различных стратегий изменения скорости обучения** с использованием инструментария PyTorch

**Рекомендуемая литература** для углубленного изучения:
* [A Recipe for Training Neural Networks](https://karpathy.github.io/2019/04/25/recipe/) - Andrej Karpathy (фундаментальное руководство по обучению нейронных сетей)
* [Bag of Tricks for Image Classification with Convolutional Neural Networks](https://arxiv.org/pdf/1812.01187v2.pdf) - обзор практических методов улучшения качества классификации изображений
* [Google's Tuning Playbook](https://github.com/google-research/tuning_playbook) - методическое руководство по планированию и проведению экспериментов

В рамках предстоящего соревнования по классификации изображений на Kaggle у вас будет возможность **применить на практике** полученные знания и освоить дополнительные методы в процессе решения реальной задачи.